In [1]:
import xarray as xr
from glob import glob
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs
import intake
import regionmask
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# see /g/data/mn51/users/gt3409/TimeSeriesExtraction/acs_catalogue_data.ipynb for building the catalogue
# This is a catalogue for ACS bias adjust data in 
# /g/data/ia39/australian-climate-service/test-data/CORDEX-CMIP6/bias-adjustment-output/AGCD-05i
col = intake.open_esm_datastore(f"/g/data/mn51/users/gt3409/nci-acs_bias_corrected.json")
col

,unique
institution_id,3
variable_id,7
table_id,1
source_id,14
downscaling_model,5
experiment_id,3
member_id,10
grid_label,1
bias_adjustment,3
time_range,141


In [3]:
# select the relevant data
variable_id = "tasmaxAdjust"
table_id= "day"
experiment_id = ["historical", "ssp370"]
bias_adjustment = "v1-r1-ACS-QME-AGCD-1960-2022"
domain = "AGCD-05i"
downscaling_model = ["CCAM-v2203-SN","BARPA-R"]

cat = col.search(variable_id = variable_id,
                 table_id= table_id,
                 experiment_id = experiment_id, 
                 bias_adjustment = bias_adjustment,
                grid_label=domain,
                downscaling_model=downscaling_model,
                )
cat.df

,institution_id,variable_id,table_id,source_id,downscaling_model,experiment_id,member_id,grid_label,bias_adjustment,time_range,path
0,BOM,tasmaxAdjust,day,ACCESS-CM2,BARPA-R,historical,r4i1p1f1,AGCD-05i,v1-r1-ACS-QME-AGCD-1960-2022,19600101-19601231,/g/data/ia39/australian-climate-service/test-d...
1,BOM,tasmaxAdjust,day,ACCESS-CM2,BARPA-R,historical,r4i1p1f1,AGCD-05i,v1-r1-ACS-QME-AGCD-1960-2022,19610101-19611231,/g/data/ia39/australian-climate-service/test-d...
2,BOM,tasmaxAdjust,day,ACCESS-CM2,BARPA-R,historical,r4i1p1f1,AGCD-05i,v1-r1-ACS-QME-AGCD-1960-2022,19620101-19621231,/g/data/ia39/australian-climate-service/test-d...
3,BOM,tasmaxAdjust,day,ACCESS-CM2,BARPA-R,historical,r4i1p1f1,AGCD-05i,v1-r1-ACS-QME-AGCD-1960-2022,19630101-19631231,/g/data/ia39/australian-climate-service/test-d...
4,BOM,tasmaxAdjust,day,ACCESS-CM2,BARPA-R,historical,r4i1p1f1,AGCD-05i,v1-r1-ACS-QME-AGCD-1960-2022,19640101-19641231,/g/data/ia39/australian-climate-service/test-d...
...,...,...,...,...,...,...,...,...,...,...,...
1822,CSIRO,tasmaxAdjust,day,EC-Earth3,CCAM-v2203-SN,ssp370,r1i1p1f1,AGCD-05i,v1-r1-ACS-QME-AGCD-1960-2022,20950101-20951231,/g/data/ia39/australian-climate-service/test-d...
1823,CSIRO,tasmaxAdjust,day,EC-Earth3,CCAM-v2203-SN,ssp370,r1i1p1f1,AGCD-05i,v1-r1-ACS-QME-AGCD-1960-2022,20960101-20961231,/g/data/ia39/australian-climate-service/test-d...
1824,CSIRO,tasmaxAdjust,day,EC-Earth3,CCAM-v2203-SN,ssp370,r1i1p1f1,AGCD-05i,v1-r1-ACS-QME-AGCD-1960-2022,20970101-20971231,/g/data/ia39/australian-climate-service/test-d...
1825,CSIRO,tasmaxAdjust,day,EC-Earth3,CCAM-v2203-SN,ssp370,r1i1p1f1,AGCD-05i,v1-r1-ACS-QME-AGCD-1960-2022,20980101-20981231,/g/data/ia39/australian-climate-service/test-d...


In [4]:
# get a list of the unique ["institution_id", "source_id", "downscaling_model", "member_id",] combinations
_list = list(cat.df.groupby(["institution_id", "source_id", "downscaling_model", "member_id",])
             .size()
             .reset_index()[["institution_id", "source_id", "downscaling_model", "member_id",]]
             .itertuples(index=False, name=None))

In [5]:
_list

[('BOM', 'ACCESS-CM2', 'BARPA-R', 'r4i1p1f1'),
 ('BOM', 'ACCESS-ESM1-5', 'BARPA-R', 'r6i1p1f1'),
 ('BOM', 'CESM2', 'BARPA-R', 'r11i1p1f1'),
 ('BOM', 'CMCC-ESM2', 'BARPA-R', 'r1i1p1f1'),
 ('BOM', 'EC-Earth3', 'BARPA-R', 'r1i1p1f1'),
 ('BOM', 'MPI-ESM1-2-HR', 'BARPA-R', 'r1i1p1f1'),
 ('BOM', 'NorESM2-MM', 'BARPA-R', 'r1i1p1f1'),
 ('CSIRO', 'ACCESS-CM2', 'CCAM-v2203-SN', 'r4i1p1f1'),
 ('CSIRO', 'ACCESS-ESM1-5', 'CCAM-v2203-SN', 'r6i1p1f1'),
 ('CSIRO', 'CESM2', 'CCAM-v2203-SN', 'r11i1p1f1'),
 ('CSIRO', 'CMCC-ESM2', 'CCAM-v2203-SN', 'r1i1p1f1'),
 ('CSIRO', 'CNRM-ESM2-1', 'CCAM-v2203-SN', 'r1i1p1f2'),
 ('CSIRO', 'EC-Earth3', 'CCAM-v2203-SN', 'r1i1p1f1')]

In [6]:
# rewrite the list in a way that bash can read
str(_list).replace(" ","").replace("(","").replace("),"," ").replace("[","").replace("]","").replace(")", "").replace("'", "")

'BOM,ACCESS-CM2,BARPA-R,r4i1p1f1 BOM,ACCESS-ESM1-5,BARPA-R,r6i1p1f1 BOM,CESM2,BARPA-R,r11i1p1f1 BOM,CMCC-ESM2,BARPA-R,r1i1p1f1 BOM,EC-Earth3,BARPA-R,r1i1p1f1 BOM,MPI-ESM1-2-HR,BARPA-R,r1i1p1f1 BOM,NorESM2-MM,BARPA-R,r1i1p1f1 CSIRO,ACCESS-CM2,CCAM-v2203-SN,r4i1p1f1 CSIRO,ACCESS-ESM1-5,CCAM-v2203-SN,r6i1p1f1 CSIRO,CESM2,CCAM-v2203-SN,r11i1p1f1 CSIRO,CMCC-ESM2,CCAM-v2203-SN,r1i1p1f1 CSIRO,CNRM-ESM2-1,CCAM-v2203-SN,r1i1p1f2 CSIRO,EC-Earth3,CCAM-v2203-SN,r1i1p1f1'

In [7]:
# Below is an example of how you could access the data using xarray.
# This is suitable for small numbers of files but was not used for the data extraction

In [8]:
df = pd.read_csv("/g/data/mn51/users/gt3409/TimeSeriesExtraction/UCL_2021_AUST_with_location_top_33.csv")
gdf = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_xy(df['Longitude'], df['Latitude']), crs=ccrs.PlateCarree())
gdf

,OBJECTID,UCL_CODE21,UCL_NAME21,SSR_CODE21,SSR_NAME21,SOS_CODE21,SOS_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,ORIG_FID,Longitude,Latitude,geometry
0,1,101001,Sydney,101,1 million or more,10,Major Urban,1,New South Wales,AUS,Australia,2194.1731,https://linked.data.gov.au/dataset/asgsed3/UCL...,0,150.966927,-33.841693,POINT (150.967 -33.842)
1,2,102001,Canberra - Queanbeyan (Queanbeyan Part),102,"250,000 to 999,999",10,Major Urban,1,New South Wales,AUS,Australia,28.7867,https://linked.data.gov.au/dataset/asgsed3/UCL...,1,149.220229,-35.366418,POINT (149.220 -35.366)
2,3,102002,Central Coast,102,"250,000 to 999,999",10,Major Urban,1,New South Wales,AUS,Australia,332.9065,https://linked.data.gov.au/dataset/asgsed3/UCL...,2,151.428768,-33.366007,POINT (151.429 -33.366)
3,4,102003,Gold Coast - Tweed Heads (Tweed Heads Part),102,"250,000 to 999,999",10,Major Urban,1,New South Wales,AUS,Australia,67.6780,https://linked.data.gov.au/dataset/asgsed3/UCL...,3,153.529595,-28.222369,POINT (153.530 -28.222)
4,5,102004,Newcastle,102,"250,000 to 999,999",10,Major Urban,1,New South Wales,AUS,Australia,296.9898,https://linked.data.gov.au/dataset/asgsed3/UCL...,4,151.677235,-32.950991,POINT (151.677 -32.951)
5,6,102005,Wollongong,102,"250,000 to 999,999",10,Major Urban,1,New South Wales,AUS,Australia,224.4836,https://linked.data.gov.au/dataset/asgsed3/UCL...,5,150.846046,-34.471490,POINT (150.846 -34.471)
6,7,111001,Albury - Wodonga (Albury Part),111,"50,000 to 99,999",11,Other Urban,1,New South Wales,AUS,Australia,74.9112,https://linked.data.gov.au/dataset/asgsed3/UCL...,6,146.948827,-36.050311,POINT (146.949 -36.050)
7,8,111002,Coffs Harbour,111,"50,000 to 99,999",11,Other Urban,1,New South Wales,AUS,Australia,77.9672,https://linked.data.gov.au/dataset/asgsed3/UCL...,7,153.095738,-30.312298,POINT (153.096 -30.312)
8,9,111003,Maitland (NSW),111,"50,000 to 99,999",11,Other Urban,1,New South Wales,AUS,Australia,122.9001,https://linked.data.gov.au/dataset/asgsed3/UCL...,8,151.587175,-32.750090,POINT (151.587 -32.750)
9,537,201001,Melbourne,201,1 million or more,20,Major Urban,2,Victoria,AUS,Australia,2880.5725,https://linked.data.gov.au/dataset/asgsed3/UCL...,538,145.077024,-37.892538,POINT (145.077 -37.893)


In [9]:
aus_gdf = gpd.read_file(
    "/g/data/ia39/aus-ref-clim-data-nci/shapefiles/data/australia/australia.shp"
)
aus_gdf["abbrevs"] = ["AUS"]
aus_gdf["NAME"] = ["Australia"]
regions = regionmask.from_geopandas(aus_gdf, names="NAME", abbrevs="abbrevs", overlap=True)


In [10]:
var="tasmaxAdjust"
institution_id, parent_model, downscaling_model, member_id = ('BOM', 'ACCESS-CM2', 'BARPA-R', 'r4i1p1f1')
year = "2015"
experiment_id = "ssp370"
filename = f"/g/data/ia39/australian-climate-service/test-data/CORDEX-CMIP6/bias-adjustment-output/AGCD-05i/{institution_id}/{parent_model}/{experiment_id}/{member_id}/{downscaling_model}/v1-r1-ACS-QME-AGCD-1960-2022/day/{var}/{var}_AGCD-05i_{parent_model}_{experiment_id}_{member_id}_{institution_id}_{downscaling_model}_v1-r1-ACS-QME-AGCD-1960-2022_day_{year}0101-{year}1231.nc"
ds = xr.open_dataset(filename, use_cftime = True,)


In [11]:
ds.sel(lon=xr.DataArray(gdf.Longitude, dims="location"), lat=xr.DataArray(gdf.Latitude, dims = 'location'), method = "nearest").tasmaxAdjust

<xarray.DataArray 'tasmaxAdjust' (time: 365, location: 32)> Size: 47kB
[11680 values with dtype=float32]
Coordinates:
  * time      (time) object 3kB 2015-01-01 12:00:00 ... 2015-12-31 12:00:00
    lat       (location) float64 256B -33.85 -35.35 -33.35 ... -12.45 -35.3
    lon       (location) float64 256B 150.9 149.2 151.4 ... 147.2 130.9 149.1
  * location  (location) int64 256B 0 1 2 3 4 5 6 7 ... 24 25 26 27 28 29 30 31
Attributes:
    long_name:      Bias-Adjusted Daily Maximum Near-Surface Air Temperature
    standard_name:  air_temperature
    units:          degC
    regrid_method:  bilinear